# Interactively Writing Web Applications

Reimagining the world of web development with Klein and Jupyter

**Moshe Zadka**

**https://cobordism.com**

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## The The Editor...And Beyond
* Interactive
* Development
* Environment

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Auto-Reloading: A Non-Solution to a Non-Problem

* Complicated and fragile
* Downtime
* Is it reloaded? But how about now?

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## What If I Told You?

I will write a web application during this talk.
At each step,
it will be usable.

There will be no reloading...just progress.

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Twisted, Meet Jupyter

* Klein on Twisted
* Twisted on Asyncioreactor
* Jupyter on Asyncio event loop

In [1]:
from twisted.internet import asyncioreactor
asyncioreactor.install()

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## The Imports

In Lisp, every file ends with `))))))))))`.

In Python, every file starts with `import import import`

In [9]:
from twisted.internet import endpoints
from twisted.web import server
from twisted.internet import reactor, defer, task
from klein import app, resource, route, Klein
import functools
import operator

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## When Endpoint Met Resource

* Klein wants to serve web requests, but doesn't know how to network
* Endpoints know how to network, but not how to send content

The romantic comedy coming soon to a conference near you!

In [3]:
endpoint = endpoints.serverFromString(reactor, "tcp:8000")
site = server.Site(resource())
endpoint.listen(site)

<Deferred at 0x7f4ecda1df10 current result: <<class 'twisted.internet.tcp.Port'> of <class 'twisted.web.server.Site'> on 8000>>

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## Let's Check It Out

In [4]:
import webbrowser
webbrowser.open("http://localhost:8000")

True

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

## If You Don't Succeed At First, Redefine Success

We are not in the *web development business*.

We are in the *incremental web development business*.

<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

In [5]:
@route("/")
def welcome(request):
    return "<html>Hello world</html>"
webbrowser.open("http://localhost:8000")

True

## No Reloading -- No Race Conditions

In [6]:
@route('/about')
def about(request):
    return 'This is the about page'
@route("/")
def welcome(request):
    return '<html>Hello world. See <a href="/about">About</a></html>'
webbrowser.open("http://localhost:8000")

True

## Modifying Routes

Added a link to an existing route -- and it changed!

In [7]:
@route('/double/<int:number>')
def double(request, number):
    result = number + number
    return f'<html>{result}</html>'
webbrowser.open("http://localhost:8000/double/12")

True

In [10]:
delayed_multiplication = functools.partial(task.deferLater, reactor, 5, operator.mul)

In [11]:
@route('/hyp_squared/<float:a>/<float:b>')
@defer.inlineCallbacks
def hyp_squared(request, a, b):
    a_squared = yield delayed_multiplication(a, a)
    b_squared = yield delayed_multiplication(b, b)
    result = a_squared + b_squared
    return f'<html>{result}</html>'
webbrowser.open("http://localhost:8000/hyp_squared/3.0/4.0")

True

In [45]:
@route('/hyp_squared/<float:a>/<float:b>')
@defer.inlineCallbacks
def hyp_squared(request, a, b):
    d_a_squared = delayed_multiplication(a, a)
    d_b_squared = delayed_multiplication(b, b)
    a_squared, b_squared = yield defer.gatherResults([d_a_squared, d_b_squared])
    result = a_squared + b_squared
    return f'<html>{result}</html>'
webbrowser.open("http://localhost:8000/hyp_squared/3.0/4.0")

True

In [12]:
delayed_negation = functools.partial(task.deferLater, reactor, 2, operator.neg)

In [13]:
@route('/distance_squared/<float:x1>/<float:y1>/<float:x2>/<float:y2>')
@defer.inlineCallbacks
def distance_squared(request, x1, y1, x2, y2):
    x, y = map(delayed_negation, (x2, y2))
    x.addCallback(operator.add, x1)
    x.addCallback(lambda t: delayed_multiplication(t, t))
    y.addCallback(operator.add, y1)
    y.addCallback(lambda t: delayed_multiplication(t, t))
    squares = yield defer.gatherResults([x, y])
    result = sum(squares)
    return f'<html>{result}</html>'
webbrowser.open("http://localhost:8000/distance_squared/2.0/1.0/5.0/5.0")

True

In [14]:
@route('/slow_distance_squared/<float:x1>/<float:y1>/<float:x2>/<float:y2>')
@defer.inlineCallbacks
def slow_distance_squared(request, x1, y1, x2, y2):
    x = x1 - (yield delayed_negation(x2))
    y = y1 - (yield delayed_negation(y2))
    x_sq = yield delayed_multiplication(x, x)
    y_sq = yield delayed_multiplication(y, y)
    result = x_sq + y_sq
    return f'<html>{result}</html>'
webbrowser.open("http://localhost:8000/slow_distance_squared/2.0/1.0/5.0/5.0")

True